In [1]:
import sqlite3
from sqlite3 import Error

In [2]:
def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn

In [3]:
def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

In [4]:
conn_norm = create_connection("insurance_management.db", delete_db = True)

create_users_table_sql = """CREATE TABLE Users (
    user_id INTEGER PRIMARY KEY,
    username VARCHAR(50),
    password VARCHAR(255),
    role VARCHAR(20),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);"""
create_table(conn_norm, create_users_table_sql)

create_customers_table_sql = """CREATE TABLE Customers (
    customer_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    customer_name VARCHAR(100),
    customer_address VARCHAR(200),
    customer_age INTEGER,
    dl_num CHAR(5),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES Users(user_id) ON DELETE SET NULL
);"""
create_table(conn_norm, create_customers_table_sql)

create_vehicles_table_sql = """CREATE TABLE Vehicles (
    vehicle_id INTEGER PRIMARY KEY,
    model VARCHAR(50),
    vehicle_age INTEGER,
    segment VARCHAR(20),
    fuel_type VARCHAR(10),
    airbags INTEGER,
    ncap_rating INTEGER,
    is_parking_camera VARCHAR(3),
    is_speed_alert VARCHAR(3),
    is_brake_assist VARCHAR(3),
    customer_id INTEGER,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id) ON DELETE CASCADE
);"""
create_table(conn_norm, create_vehicles_table_sql)

create_insurance_table_sql = """CREATE TABLE Insurance (
    policy_id VARCHAR(30) PRIMARY KEY,
    subscription_length NUMERIC,
    claim_status VARCHAR(30),
    vehicle_id INTEGER,
    customer_id INTEGER,
    FOREIGN KEY (vehicle_id) REFERENCES Vehicles(vehicle_id) ON DELETE CASCADE,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id) ON DELETE CASCADE
);"""
create_table(conn_norm, create_insurance_table_sql)

create_claims_table_sql = """CREATE TABLE Claims (
    claim_id INTEGER PRIMARY KEY,
    policy_id VARCHAR(30),
    customer_id INTEGER,
    claim_date DATE NOT NULL,
    claim_amount DECIMAL(10, 2) NOT NULL,
    claim_status VARCHAR(20) NOT NULL,
    resolution_date DATE,
    FOREIGN KEY (policy_id) REFERENCES Insurance(policy_id) ON DELETE CASCADE,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id) ON DELETE CASCADE
);"""
create_table(conn_norm, create_claims_table_sql)

create_agents_table_sql = """CREATE TABLE Agents (
    agent_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    agent_name VARCHAR(100) NOT NULL,
    contact_number VARCHAR(15),
    email VARCHAR(100) NOT NULL UNIQUE,
    FOREIGN KEY (user_id) REFERENCES Users(user_id) ON DELETE SET NULL
);"""
create_table(conn_norm, create_agents_table_sql)

create_payments_table_sql = """CREATE TABLE Payments (
    payment_id INTEGER PRIMARY KEY,
    policy_id VARCHAR(30),
    customer_id INTEGER,
    payment_date DATE NOT NULL,
    amount DECIMAL(10, 2) NOT NULL,
    payment_method VARCHAR(50),
    payment_status VARCHAR(20) NOT NULL,
    FOREIGN KEY (policy_id) REFERENCES Insurance(policy_id) ON DELETE CASCADE,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id) ON DELETE CASCADE
);"""
create_table(conn_norm, create_payments_table_sql)

create_police_table_sql = '''
    CREATE TABLE Police (
        police_id INTEGER PRIMARY KEY,
        user_id INTEGER NOT NULL,
        police_name TEXT NOT NULL,
        police_station TEXT NOT NULL,
        police_contact TEXT NOT NULL,
        FOREIGN KEY (user_id) REFERENCES Users (id)
    );'''
create_table(conn_norm, create_police_table_sql)

conn_norm.commit()
conn_norm.close()

In [5]:
def insert_users(conn, values):
    sql = ''' INSERT INTO Users (
              username, password, role)
              VALUES (?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

def insert_customers(conn, values):
    sql = ''' INSERT INTO Customers (
              user_id, customer_name, customer_address, customer_age, dl_num)
              VALUES (?, ?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

def insert_vehicles(conn, values):
    sql = ''' INSERT INTO Vehicles (
              model, vehicle_age, segment, fuel_type, airbags, ncap_rating, 
              is_parking_camera, is_speed_alert, is_brake_assist, customer_id)
              VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

def insert_insurance(conn, values):
    sql = ''' INSERT INTO Insurance (
              policy_id, subscription_length, claim_status, vehicle_id, customer_id)
              VALUES (?, ?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

def insert_claims(conn, values):
    sql = ''' INSERT INTO Claims (
              policy_id, customer_id, claim_date, claim_amount,
              claim_status, resolution_date)
              VALUES (?, ?, ?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

def insert_agents(conn, values):
    sql = ''' INSERT INTO Agents (
              user_id, agent_name, contact_number, email) 
              VALUES (?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

def insert_payments(conn, values):
    sql = ''' INSERT INTO Payments (
              policy_id, customer_id, payment_date, amount, payment_method, payment_status)
              VALUES (?, ?, ?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

def insert_police(conn, values):
    sql = ''' INSERT INTO Police (
              user_id, police_name, police_station, police_contact)
              VALUES (?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

In [6]:
def check_existing_email(conn, email):
    sql = ''' SELECT COUNT(*) FROM Agents WHERE email = ? '''
    cur = conn.cursor()
    cur.execute(sql, (email,))
    return cur.fetchone()[0] > 0

In [7]:
import csv
from faker import Faker

fake = Faker()
conn_norm2= sqlite3.connect('insurance_management.db')
csv_file=open('insurance_claims_data.csv')
csv_reader = csv.DictReader(csv_file)
for row in csv_reader:
    user_role = fake.random_element(elements=("customer", "agent", "police"))
    users_table_values = (fake.user_name(), fake.password(), user_role)
    userID = insert_users(conn_norm2, users_table_values)
    if user_role == "customer":
        customers_table_values = (userID, fake.name(), fake.address(), row['customer_age'],fake.random_number(digits=5))
        customerID = insert_customers(conn_norm2, customers_table_values)
        vehicles_table_values = (row['model'], row['vehicle_age'], row['segment'], row['fuel_type'], row['airbags'],
                row['ncap_rating'], row['is_parking_camera'], row['is_speed_alert'], row['is_brake_assist'], customerID)
        vehicleID = insert_vehicles(conn_norm2, vehicles_table_values)
        policyID = row['policy_id']
        insurance_table_values = (policyID, row['subscription_length'], row['claim_status'], vehicleID, customerID)
        insert_insurance(conn_norm2, insurance_table_values)
        claims_table_values = (policyID,customerID,fake.date_this_year(),fake.random_number(digits = 5),
                               row['claim_status'],fake.date_this_year())
        insert_claims(conn_norm2, claims_table_values)
        payment_method = fake.random_element(elements = ("card", "paypal", "others"))
        payment_status = fake.random_element(elements = ("success", "in progress", "declined"))
        payments_table_values = (policyID,customerID,fake.date_this_year(),fake.random_number(digits = 5), payment_method,
                                 payment_status)
        insert_payments(conn_norm2, payments_table_values)
    elif user_role == "agent":
        agent_email = fake.email()
        while check_existing_email(conn_norm2, agent_email):
            agent_email = fake.email()
        agent_table_values = (userID, fake.name(), fake.random_number(digits=5), agent_email)
        insert_agents(conn_norm2, agent_table_values)
    elif user_role == "police":
        police_table_values = (userID, fake.name(), fake.address(), fake.random_number(digits=5))
        insert_police(conn_norm2, police_table_values)

conn_norm2.commit()
conn_norm2.close()

In [8]:
import pandas as pd
conn = sqlite3.connect('insurance_management.db')
tables = ["Users", "Customers", "Vehicles", "Insurance", "Claims", "Agents", "Payments", "Police"]

for table in tables:
    query = f"SELECT * FROM {table}"
    df = pd.read_sql_query(query, conn)
    csv_file_path = f"{table}.csv"
    df.to_csv(csv_file_path, index=False)
    print(f"Table {table} exported to {csv_file_path}")

conn.close()
print("Data exported successfully to CSV files.")

Table Users exported to Users.csv
Table Customers exported to Customers.csv
Table Vehicles exported to Vehicles.csv
Table Insurance exported to Insurance.csv
Table Claims exported to Claims.csv
Table Agents exported to Agents.csv
Table Payments exported to Payments.csv
Table Police exported to Police.csv
Data exported successfully to CSV files.
